In [1]:
import sys
import os

sys.path.append('../')

%load_ext autoreload
%autoreload 2
%matplotlib inline


os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../data')

train_df = shuffle(train_df)
valid_df = shuffle(valid_df)

print(train_df.head())


                                                image                   id  \
1983                   ../data/Train/CameraRGB/72.png                   72   
299   ../data/Train/CameraRGB/episode_0002_000134.png  episode_0002_000134   
518                   ../data/Train/CameraRGB/536.png                  536   
1861                  ../data/Train/CameraRGB/214.png                  214   
944   ../data/Train/CameraRGB/episode_0002_000277.png  episode_0002_000277   

                                                label  
1983                   ../data/Train/CameraSeg/72.png  
299   ../data/Train/CameraSeg/episode_0002_000134.png  
518                   ../data/Train/CameraSeg/536.png  
1861                  ../data/Train/CameraSeg/214.png  
944   ../data/Train/CameraSeg/episode_0002_000277.png  


In [4]:
from models.segnet import model_segnetVGG16

model = model_segnetVGG16(3, image_shape=(480, 480, 3))
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 480, 480, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 480, 480, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 480, 480, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 240, 240, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 240, 240, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 240, 240, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 120, 120, 128)     0         
__________

In [5]:
from gen.generators import train_and_lab_gen_func

BATCH_SIZE = 8
model_dir = '../saved_models/segnet/'

train_gen = train_and_lab_gen_func(train_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE)
valid_gen = train_and_lab_gen_func(valid_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE)


In [6]:
model.load_weights('{}/model.hdf5'.format(model_dir))

In [7]:
from train import weighted_categorical_crossentropy
from keras.utils.training_utils import multi_gpu_model

weights = np.array([10, 5, 1])

# gpus = 2

# model_gpu = multi_gpu_model(model, gpus)
model.compile(loss=weighted_categorical_crossentropy(weights),
              optimizer='adam',
              metrics=['accuracy'])


In [8]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model, 
                   train_gen, 
                   valid_gen, 
                   training_size=m, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=500,
                   gpus = 1)

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/keras/engine/training.py:2095: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


## Ignore next message from keras, values are replaced anyways
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 500 images
Found 0 images belonging to 0 classes.
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 500 images
## Ignore next

Process Process-2:
Traceback (most recent call last):
  File "/home/faisal/anaconda3/envs/ai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/faisal/anaconda3/envs/ai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/keras/utils/data_utils.py", line 663, in _data_generator_task
    generator_output = next(self._generator)
  File "../gen/generators.py", line 84, in train_and_lab_gen_func
    for (x, _), (y, _) in zip(train_rgb_gen, train_lab_gen):
  File "/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/keras/preprocessing/image.py", line 834, in __next__
    return self.next(*args, **kwargs)
  File "/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/keras/preprocessing/image.py", line 1189, in next
    return self._get_batches_of_transformed_samples(index_array)
  File "/home/faisal/anaconda3/envs/ai/l

KeyboardInterrupt: 

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('../saved_models/segnet/model_saved.h5')

In [ ]:
#valid_gen = train_and_lab_gen_func(valid_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE * gpus)
#test_gen = train_and_lab_gen_func(test_df, image_size=(600, 800),  target_size=(480, 480),  batch_size = BATCH_SIZE * gpus)